<a href="https://colab.research.google.com/github/studboii/ML-Colab_Demos/blob/main/Notebooks/Kabhi_Froot_Kabhi_Shahad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
!pip install -qq gdown

In [ ]:
! gdown -O Fruits.zip https://drive.google.com/uc?id=159netVOtnpAABYJHHVuulH7tWUC6JJ4D

In [ ]:
#We use 7z to unzip our file. From my experience, 7z is usually faster and has cleaner output
! 7z x /content/Fruits.zip

In [ ]:
pth  = "/content/fruits-360/Training"

In [ ]:
FruitsDls = DataBlock(blocks=(ImageBlock, CategoryBlock), get_items=get_image_files, splitter=RandomSplitter(valid_pct=0.2, seed=42), get_y=parent_label, item_tfms=Resize(100))
#This is our fruit dataloader object In the next bit of code, I'll specify the path and hopefully the model works at recognizing which fruit is which


In [ ]:
FruitsDls = FruitsDls.new(item_tfms=Resize(110, ResizeMethod.Pad, pad_mode='zeros'))
dls = FruitsDls.dataloaders(pth)

In [ ]:
dls.valid.show_batch(max_n = 14, nrows = 2)
# Show me two rows of a random batch

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate,path="/content/MyModel/") #make a learner for our dataloader
learn.fine_tune(5) #have 4 runs of training data to optimse our model

In [ ]:
#If it doesnt't save the model, run the next part to save and download it
learn.export("/content/MyModel/FruitModelResnet18.pkl")

In [ ]:
#Now we plot our top losses to figure out where we might be a wittwe bit fucky wucky
# interp.plot_top_losses(9, nrows=9)
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(5, nrows=1)

In [ ]:
img = PILImage.create("/content/fruits-360/Test/Pear Red/200_100.jpg")
stuff = learn.predict(img)
# print("Probability that this is a cat: "+str(probs[1].item()*100))
print(str(stuff))
print("Item is a: "+str(stuff[0]))

In [ ]:
learn.show_results()

In [ ]:
learn.dls.vocab

In [ ]:
WasaLearner = load_learner("/content/MyModel/FruitModelResnet18.pkl")

In [ ]:
# Making all the widgets we need
btn_upload = widgets.FileUpload() #Upload Widget
out_pl = widgets.Output() #View What you uploaded
btn_runner  = widgets.Button(description="Wat could it be lmao") #Classify button
lbl_pred = widgets.Label() #Answer label

In [ ]:
# def Button_Saaf():
#     btn_upload.value.clear()
#     btn_upload._counter = 0

def on_click_run(change):
    img = PILImage.create(btn_upload.data[0])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = WasaLearner.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_runner.on_click(on_click_run)

In [ ]:
# Button_Saaf()
btn_upload.value.clear()
btn_upload._counter = 0
VBox([widgets.Label('Select your fruit'), btn_upload, btn_runner, out_pl, lbl_pred])